# Notebook 3: Analyzing Existing DA Paleoclimate Reconstructions

## Introduction

In this notebook, we'll look at several paleoclimate reconstructions that were made with data assimilation. Two of them cover the Common Era and two cover the Holocene or beyond. Some data for these reconstructions has already been downloaded. Later, if you want to look for other variables, some of the reconstructions have additional variables that can be found at the links below.

| Reconstruction | Temporal Coverage | Variables | Data Link | Code Link |
| --- | --- | --- | --- | --- |
| Last Millennium Reanalysis (LMR 2.0) | 0 - 2000 CE | Temp, Precip, and More | https://www.ncei.noaa.gov/access/paleo-search/study/27850 | https://github.com/modons/LMR |
| Paleo Hydrodynamics Data Assimilation product (PHYDA) | 1 - 2000 CE | Temp and Precip | https://zenodo.org/record/1198817 | https://github.com/njsteiger/PHYDA |
| Last Glacial Maximum reanalysis (LGMR) | 0 - 24000 yr BP | Temp and d18O | https://www.ncei.noaa.gov/access/paleo-search/study/33112 | https://github.com/JonKing93/DASH |
| Holocene Reconstruction | 0 - 12000 yr BP | Temp | https://zenodo.org/record/6426332 | https://github.com/Holocene-Reconstruction/Holocene-code |

Each reconstruction is stored in NetCDF files. To see what files I already downloaded, visit the da_workshop/data/ folder on Google Drive.

In this notebook, you'll be loading one of these reconstructions and making figures of global mean temperature as well as spatial changes for different time periods.

## 1. Setting up Google Colab

First, let's install some libraries.

In [ ]:
# Install cartopy and cftime
!pip install cartopy
!pip install cftime

To mount Google Drive, run the code below and, when prompted, allow it to connect with your Google account.

In [ ]:
# Mount Google Drive locally
from google.colab import drive
drive.mount('/content/drive')

## 2. Importing necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.util as cutil
import cartopy.feature as cfeature
import xarray as xr
import cftime
plt.style.use('ggplot')  # This sets the plotting style for the figures we'll be making later.

## 3. Reconstructed variables

Different reconstructions focus on different variables. Some of these variables are listed in the table below. (There are also time, lat, and lon variables in each file.)

After going through the notebook, feel free to load a different variable to do another analysis. Additional variables may be downloaded at the links at the top of this notebook.

### LMR

I've downloaded some variables for LMR. Files containing other variables can be downloaded from the link at the top of the page.

| Variable name | Dimensions | Description | File name |
| --- | --- | --- | --- |
| air | time-MCrun-lat-lon | Temperature, ensemble means | air_MCruns_ensemble_mean_LMRv2.0.nc |
| sst | time-MCrun-lat-lon | Sea surface temperature, ensemble means | sst_MCruns_ensemble_mean_LMRv2.0.nc |
| pdsi | time-MCrun-lat-lon | Self-calibrated Palmer Drought Severity Index, ensemble means | pdsi_MCruns_ensemble_mean_LMRv2.0.nc |
| prate | time-MCrun-lat-lon | Precipitation, ensemble means | prate_MCruns_ensemble_mean_LMRv2.0.nc |
| pr_wtr | time-MCrun-lat-lon | Precipitable water for entire atmosphere, ensemble means | pr_wtr_MCruns_ensemble_mean_LMRv2.0.nc |
| hgt500 | time-MCrun-lat-lon | 500 hPa geopotential height, ensemble means | hgt500_MCruns_ensemble_mean_LMRv2.0.nc |
| prmsl | time-MCrun-lat-lon | Pressure at mean sea level, ensemble means | prmsl_MCruns_ensemble_mean_LMRv2.0.nc |
| gmt | time-MCrun-members | Global mean temperature | gmt_MCruns_ensemble_full_LMRv2.0.nc |
| nhmt | time-MCrun-members | Northern Hemisphere mean temperature | nhmt_MCruns_ensemble_full_LMRv2.0.nc |
| shmt | time-MCrun-members | Southern Hemisphere mean temperature | shmt_MCruns_ensemble_full_LMRv2.0.nc |

### PHYDA

For PHYDA, all variables have variables for ensemble-mean (mn), ensemble standard deviation (sg), and 5th, 50th, and 95th percentiles (pc). Replace the asterisk below with "mn", "sg", or "pc" to load these variables. (The "pc" variables have an additional dimension of size 3.) Additional variables are also stored in the file. June-August and December-February means can be downloaded at the link above.

| Variable name | Dimensions | Description | File name |
| --- | --- | --- | --- |
| tas_* | time-lat-lon | Temperature | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc |
| pdsi_* | time-lat-lon | PDSI (land only) | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc |
| spei_* | time-lat-lon | SPEI (land only) | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc |
| Nino_12_* | months | Nino 1+2 | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc |
| Nino_3_* | months | Nino 3 | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc |
| Nino_3.4_* | months | Nino 3.4 | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc |
| Nino_4_* | months | Nino 4 | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc |
| gmt_* | time | Global mean temperature | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc |
| amo_* | time | AMO (NASST index) | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc | da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc |

### LGMR

I've downloaded some variables for LGMR. Files containing other variables can be downloaded from the link at the top of the page. For each variable listed below, append "_std" to the end of the variable to load the ensemble standard deviation instead of the mean.

| Variable name | Dimensions | Description | File name |
| --- | --- | --- | --- |
| sat | time-lat-lon | Temperature, ensemble mean | LGMR_SAT_climo.nc |
| d18Op | time-lat-lon | d18O of precipitation, ensemble mean | LGMR_d18Op_climo.nc |
| gmst | time | Global mean surface temperature, ensemble mean | LGMR_GMST_climo.nc |

### Holocene Reconstruction

A subset of the variables stored in the Holocene Reconstruction is listed below.

| Variable name | Dimensions | Description | File name |
| --- | --- | --- | --- |
| recon_tas_mean | time-lat-lon | Temperature, ensemble mean | holocene_reconstruction.nc |
| recon_tas_ens | time-selected_ens-lat-lon | Temperature, 100 ensemble members | holocene_reconstruction.nc |
| recon_tas_global_mean | time-ens | Global mean temperature, all ensemble members | holocene_reconstruction.nc |
| prior_tas_mean | time-lat-lon | Temperature of the prior, ensemble mean | holocene_reconstruction.nc |
| prior_tas_global_mean | time-ens | Global mean temperature of the prior, all ensemble members | holocene_reconstruction.nc |
| proxy_values | time-proxies | The data of all proxy records, nearest-neighbor interpolated to decadal resolution | holocene_reconstruction.nc |
| proxy_uncertainty | proxies | Uncertainty values for all proxies | holocene_reconstruction.nc |
| proxy_metadata | proxies-8 | Proxy metadata: datasetname, TSid, lat, lon, seasonality_months, seasonality_general, median age resolution, collection | holocene_reconstruction.nc |
| options | 26 | The options used for this experiment | holocene_reconstruction.nc |
| proxies_assimilated | time-proxies | Information about whether each proxy was assimilated (1) or omitted (0) in each timestep | holocene_reconstruction.nc |

## 4. Loading a paleoclimate reconstruction

Now, let's load one of the paleoclimate reconstructions. Below, there are code cells to load spatial temperature in each of the four reconstructions. **To load the data, only run one of these.** If you run more than one, only the last one will be loaded.

Note: The "xarray" library has additional functionality for working data, so feel free to explore that after the workshop: https://docs.xarray.dev/en/stable/getting-started-guide/index.html. In this notebook, we'll just extract the data as arrays. I think this leads to more readable code. Your milage may vary.

### Option 1: LMR

In [ ]:
#%% LMR
dataset_chosen = 'LMR'
data_xarray = xr.open_dataset('/content/drive/MyDrive/da_workshop/data/air_MCruns_ensemble_mean_LMRv2.0.nc')
tas_mean = np.mean(data_xarray['air'].values,axis=1)  # This variable has 20 ensemble values, but we're averaging them together here. Feel free to look at the ensemble range, though.
time     = data_xarray['time'].values
lat      = data_xarray['lat'].values
lon      = data_xarray['lon'].values
data_xarray.close()
years = np.array([i.year for i in time])
ages = 1950 - years

### Option 2: PHYDA

In [ ]:
#%% PHYDA
dataset_chosen = 'PHYDA'
data_xarray = xr.open_dataset('/content/drive/MyDrive/da_workshop/data/da_hydro_AprMar_r.1-2000_d.05-Jan-2018.nc')
tas_mean = data_xarray['tas_mn'].values
years    = data_xarray['time'].values
lat      = data_xarray['lat'].values
lon      = data_xarray['lon'].values
data_xarray.close()
ages = 1950 - years

### Option 3: LGMR

In [ ]:
#%% LGMR
dataset_chosen = 'LGMR'
data_xarray = xr.open_dataset('/content/drive/MyDrive/da_workshop/data/LGMR_SAT_climo.nc')
tas_mean = data_xarray['sat'].values
ages     = data_xarray['age'].values
lat      = data_xarray['lat'].values
lon      = data_xarray['lon'].values
data_xarray.close()

### Option 4: Holocene Reconstruction

In [ ]:
#%% Holocene_DA
dataset_chosen = 'Holocene_DA'
data_xarray = xr.open_dataset('/content/drive/MyDrive/da_workshop/data/holocene_reconstruction.nc')
tas_mean = data_xarray['recon_tas_mean'].values
ages     = data_xarray['ages'].values
lat      = data_xarray['lat'].values
lon      = data_xarray['lon'].values
data_xarray.close()
print(np.shape(tas_mean))

**Remember, use only one of the four datasets above. If you run more than one, only the last one you ran will be loaded.**

To see what's in the file you loaded, show the xarray. You can click on parts of the table below to get more information.

In [ ]:
# Show what's in the data
data_xarray

## 5. Plotting results

Now that the data is loaded, let's make some figures. The tas_mean variable is a 3D array with dimensions of age-lat-lon.

### 5.1. Time series

Start by plotting a global mean. The function below calculates the mean value over a specified region. The region is specified as a list with four values: [lon_min, lon_max, lat_min, lat_min]. Longitude in all four of these reconstruction goes from 0-360 (make sure to check longitude values when opening a new dataset).

In [ ]:
# A function to computes a regional-mean from a time-lat-lon variable
def spatial_mean(variable,region_bounds):
    #
    i_selected = np.where((lon >= region_bounds[0]) & (lon <= region_bounds[1]))[0]
    j_selected = np.where((lat >= region_bounds[2]) & (lat <= region_bounds[3]))[0]
    print('Computing spatial mean. lats='+str(lat[j_selected[0]])+'-'+str(lat[j_selected[-1]])+', lons='+str(lon[i_selected[0]])+'-'+str(lon[i_selected[-1]])+'.  Points are inclusive.')
    #
    lat_weights = np.cos(np.radians(lat))
    variable_zonal = np.nanmean(variable[:,:,i_selected],axis=2)
    variable_mean = np.average(variable_zonal[:,j_selected],axis=1,weights=lat_weights[j_selected])
    #
    return variable_mean

Call the function above to compute a spatial mean, then plot it.

In [ ]:
# Make a global-mean plot
region_bounds = [0,360,-90,90]
tas_global_mean = spatial_mean(tas_mean,region_bounds)

# Plot the main composite
f,ax1 = plt.subplots(1,1,figsize=(12,6))
ax1.plot(ages,tas_global_mean,linewidth=3)
ax1.set_xlim(max(ages),min(ages))
ax1.set_ylabel('$\Delta$T ($^\circ$C)',fontsize=16)
ax1.set_xlabel('Age (yr BP)',fontsize=16)
ax1.set_title('Mean $\Delta$T ($^\circ$C) for '+dataset_chosen+', region: region='+str(region_bounds),fontsize=18,loc='center')
plt.show()

**Try this:** Change the region_bounds variable above to plot the temperature reconstruction for a different region.

### 5.2. Maps and zonal means

Data assimilation creates spatial reconstructions, so we can also make figures showing spatial temperature anomalies for different time periods. The function below makes two figures. One shows a map and the other shows a zonal mean figure of temperature differences.

In [ ]:
# A function to make a map of differences between ages
def map_temp_anom(ages_anom=[5500,6500],ages_ref=[0,1000]):
    #
    # Compute the difference between the periods specified above.
    ind_anom = np.where((ages >= ages_anom[0]) & (ages <= ages_anom[1]))[0]
    ind_ref  = np.where((ages >= ages_ref[0])  & (ages <= ages_ref[1]))[0]
    print('Data points in anomaly period:  ',len(ind_anom))
    print('Data points in reference period:',len(ind_ref))
    if (len(ind_anom) == 0) or (len(ind_ref) == 0): print(' === WARNING: NO DATA FOR ONE OR BOTH PERIODS. MAP WILL BE BLANK. ===')
    tas_change = np.mean(tas_mean[ind_anom,:,:],axis=0) - np.mean(tas_mean[ind_ref,:,:],axis=0)
    tas_change_zonal = np.mean(tas_change,axis=1)
    #
    # Make a map of changes
    plt.figure(figsize=(12,8))
    ax1 = plt.subplot2grid((1,1),(0,0),projection=ccrs.Robinson()); ax1.set_global()
    values_range = np.arange(-1,1.1,.1)
    tas_change_cyclic,lon_cyclic = cutil.add_cyclic_point(tas_change,coord=lon)
    map1 = ax1.contourf(lon_cyclic,lat,tas_change_cyclic,values_range,extend='both',cmap='bwr',transform=ccrs.PlateCarree())
    colorbar1 = plt.colorbar(map1,orientation='horizontal',ax=ax1,fraction=0.08,pad=0.02)
    colorbar1.set_label('$\Delta$T ($^\circ$C)',fontsize=16)
    ax1.set_title('$\Delta$T ($^\circ$C) for '+dataset_chosen+', ages: anom='+str(ages_anom)+', ref='+str(ages_ref),loc='center',fontsize=16)
    ax1.coastlines()
    ax1.add_feature(cfeature.LAKES,facecolor='none',edgecolor='k')
    ax1.gridlines(color='k',linewidth=1,linestyle=(0,(1,5)))
    ax1.spines['geo'].set_edgecolor('black')
    plt.show()
    #
    # Make a zonal mean figure of the changes
    f,ax1 = plt.subplots(1,1,figsize=(10,10))
    ax1.plot(tas_change_zonal,lat,linewidth=3)
    ax1.axvline(x=0,color='gray',alpha=1,linestyle=':',linewidth=2)
    ax1.legend(loc='lower right')
    ax1.set_ylim(-90,90)
    ax1.set_xlabel('$\Delta$T ($^\circ$C)',fontsize=16)
    ax1.set_ylabel('Latitude ($^\circ$)',fontsize=16)
    ax1.set_title('Zonal-mean $\Delta$T ($^\circ$C), ages: anom='+str(ages_anom)+', ref='+str(ages_ref),loc='center',fontsize=16)
    plt.show()

Before making the figures, double-check the ages in your dataset by printing the beginning and end of the age variable.

In [ ]:
print('First 5 ages:',ages[:5])
print('Last 5 ages:',ages[-5:])

Now, uncomment one of the lines below and make the figures. Feel free to set different values to plot different times.

If you try to plot a time that doesn't have any data, you'll get a blank map.

**If you get an error, make sure you set the ages_anom and ages_ref variables.**

In [ ]:
# Make a map of differences between ages
#ages_anom = [-50,-40];     ages_ref = [95,105]  # 1990-2000 CE vs 1845-1855 CE
#ages_anom = [5500,6500];   ages_ref = [0,1000]  # 6 ka vs 0 ka
#ages_anom = [20500,21500]; ages_ref = [0,1000]  # 21 ka vs 0 ka

map_temp_anom(ages_anom,ages_ref)

## 6. Uncertainty in reconstructions

So far, we've only looked at the mean reconstruction in this notebook. However, a great feature of data assimilation is that the recosntruction is an *ensemble*. The ensemble helps quantify the reconstruction's uncertainty.

When looking at a reconstruction, always check out this ensemble. If the quantity that you're interested in has a wide ensemble range, you should keep that in mind: a wide ensemble range indicates increased uncertainty.

All four of the reconstructions have ensembles. However, to keep file sizes smaller, they may be saved in different ways.

In this section, we'll load the Holocene reconstruction and plot the **global-mean temperature ensemble** as well as the **prior ensemble**.

In [ ]:
#%% Holocene_DA
dataset_chosen = 'Holocene_DA'
data_xarray = xr.open_dataset('/content/drive/MyDrive/da_workshop/data/holocene_reconstruction.nc')
recon_tas_global_mean = data_xarray['recon_tas_global_mean'].values
prior_tas_global_mean = data_xarray['prior_tas_global_mean'].values
ages                  = data_xarray['ages'].values
data_xarray.close()

The two variables:
- **prior_tas_global_mean:** Global mean temperature in the model "prior," which is the initial collection of model states, before any data assimilation is preformed.
- **recon_tas_global_mean:** Global mean temperature for the final reconstruction. This is sometimes called the "posterior."

Both variables give global mean temperature for all 1200 time steps and all 1002 ensemble members. Check the shape to verify:

In [ ]:
print(recon_tas_global_mean.shape)

Make a plot showing global mean temperature in the prior and reconstruction. Specifically, we'll plot the mean and 95% range for the ensembles.

In [ ]:
# Plot global mean temperature in the prior and final reconstruction
f,ax1 = plt.subplots(1,1,figsize=(12,6))
ax1.fill_between(ages,np.percentile(prior_tas_global_mean,2.5,axis=1),np.percentile(prior_tas_global_mean,97.5,axis=1),color='gray',alpha=0.25)
ax1.fill_between(ages,np.percentile(recon_tas_global_mean,2.5,axis=1),np.percentile(recon_tas_global_mean,97.5,axis=1),color='tab:blue',alpha=0.5)
ax1.plot(ages,np.nanmean(prior_tas_global_mean,axis=1),color='gray',    linewidth=3,label='Prior')
ax1.plot(ages,np.nanmean(recon_tas_global_mean,axis=1),color='tab:blue',linewidth=3,label='Reconstruction')
ax1.axhline(y=0,color='k',linewidth=1,linestyle='dashed',alpha=0.5)
ax1.legend(loc='lower right',ncol=2,fontsize=16)
ax1.set_xlim(12000,0)
ax1.set_ylim(-3.5,.5)
ax1.set_ylabel('$\Delta$T ($^\circ$C)',fontsize=16)
ax1.set_xlabel('Age (yr BP)',fontsize=16)
ax1.set_title('Global-mean temperature anomaly from reconstruction and prior ($^\circ$C)',fontsize=18,loc='center')
plt.show()

Two points to consider:
- Keep in mind that the ensemble range can differ for every time step, location, and variable.
- Comparing the prior to the reconstruction shows how much the reconstruction was shaped by assimilating proxy data.

## 7. Keep Exploring!

Now go back and try:
- Plotting time series for different regions
- Plotting maps for different time periods
- Loading different reconstructions
- Loading different variables

Use the map to find interesting regions for the time series plots and vice versa.

Once you find something interesting, save a figure or two.

**This is the end of the materials for Day 1 of the Paleo DA Workshop. Join us on our Zoom link at 9 AM PDT tomorrow (time zone permitting) for more talks and data analysis. Thanks!**